# Análise Exploratória dos Dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import squarify
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

### Lendo e Preparando os Dados

In [ ]:
df_train = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

In [ ]:
df_train['df_type'] = 'train'
target = df_train[['Id', 'Target']]
df_test['df_type'] = 'test'
df_all = pd.concat([df_train.drop(columns=['Target']), df_test])

## Imputações

In [ ]:
def fill_nan_distribution(df, column_name):
    data_column = df[column_name]
    probs = data_column.value_counts(normalize=True).sort_values() # Gerando percentual dos valores não nulos
    tot_nan = data_column.isna().sum()
    index_nan = df.loc[data_column.isna(), column_name].index
    sample_space = list(probs.index)
    new_values = np.random.choice(
        sample_space, size=tot_nan, p=probs
    )
    return pd.Series(data=new_values, index=index_nan)

In [ ]:
df_all.loc[df_all.rez_esc.isna(), 'rez_esc'] = fill_nan_distribution(df_all, 'rez_esc')
df_all.loc[df_all.v18q1.isna(), 'v18q1'] = fill_nan_distribution(df_all, 'v18q1')
df_all.loc[df_all.v2a1.isna(), 'v2a1'] = df_all.v2a1.mean()
df_all.loc[df_all.SQBmeaned.isna(), 'SQBmeaned'] = df_all.SQBmeaned.mean()
df_all.loc[df_all.meaneduc.isna(), 'meaneduc'] = df_all.meaneduc.mean()

## Análise Exploratória dos Dados

In [ ]:
cols = ['Id', 'computer', 'television', 'mobilephone', 'rooms', 'tamhog', 'escolari', 'rez_esc', 'bedrooms', 'meaneduc', 'hogar_total', 'hogar_mayor', 'hogar_adul', 'hogar_nin', 'male', 'female', 'cielorazo', 'v2a1', 'hacdor', 'hacapo', 'sanitario1']
df_eda = df_all[cols].copy()

In [ ]:
def pie_chart(vals, labels, ax, title='', **kwargs):
    my_circle=plt.Circle( (0,0), 0.65, color='white')
    ax.pie(vals, labels=labels, autopct='%1.1f%%', pctdistance=.8, **kwargs)
    ax.add_artist(my_circle)
    ax.set_title(title, fontsize=14)
    return ax

def plot_pie_percent(df, col, ax, title=''):
    df_comp = (
        df
        .groupby(col, as_index=False)
        .count()
        .loc[:, [col, 'Id']]
        .rename(columns={'Id': 'qtd'})
        .replace({0: 'não', 1: 'sim'})
    )
    pie_chart(df_comp.qtd, df_comp[col], ax, title)

def add_porcentagem(ax, total_val, w = .8, h = 100):
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * (p.get_height()/total_val))
        x = p.get_x() + p.get_width() - w
        y = p.get_y() + p.get_height() + h
        ax.annotate(percentage, (x, y), size = 12)
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 6))

plot_pie_percent(df_eda, 'computer', ax[0], 'Famílias que possuem computador')
plot_pie_percent(df_eda, 'television', ax[1], 'Famílias que possuem TV')
plot_pie_percent(df_eda, 'mobilephone', ax[2], 'Famílias que possuem celular')

plt.tight_layout()
plt.show()

- Analisando os gráficos de pizza acima podemos notar que 88,9% das famílias não possuem computador, 71,2% das famílias não possuem televisão mas, 97,7% das famílias possuem celular.

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

fig.suptitle('Casas por Número de Cômodos', fontsize=16)
df_rooms = (
    df_eda
    .groupby('rooms', as_index=False)
    .count()
    .loc[:, ['rooms', 'Id']]
    .rename(columns={'Id': 'qtd'})
)
ax = sns.barplot(
    x=df_rooms.rooms, y=df_rooms['qtd'], data=df_rooms, palette="Blues_d"
)
add_porcentagem(ax, df_rooms.qtd.sum())

plt.show()

- O gráfico acima apresenta o precentual de casas por número de cômodos. Podemos notar que a grande maioria das casas presentes na base de dados possuem 4 e 5 cômodos (16,7% e 30,7% respectivamente), e uma pequena parte do conjunto de observações possuem mais de 9 cômodos.

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

fig.suptitle('Casas por Número de Quartos de Dormir', fontsize=16)
df_beds = (
    df_eda
    .groupby('bedrooms', as_index=False)
    .count()
    .loc[:, ['bedrooms', 'Id']]
    .rename(columns={'Id': 'qtd'})
)
ax = sns.barplot(
    x=df_beds.bedrooms, y=df_beds['qtd'], data=df_beds, palette="YlOrBr_d"
)
add_porcentagem(ax, df_beds.qtd.sum(), w=.6, h=100)
plt.show()

- O gráfico acima apresenta as casas por quantidade de quartos de dormir. Neste gráfico é possível observar que 40% das casas possuem 3 quartos para dormir, e poucos mais de 4% das casas possuem mais de 5 quartos.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].boxplot(df_eda.escolari)
ax[0].set_title('Anos de Escolaridade', pad=20, fontsize=16)

ax[1].boxplot(df_eda.meaneduc)
ax[1].set_title('Anos Médio de Educação para Adultos', pad=20, fontsize=16)

plt.show()

- O gráfico acima apresenta o boxplot do ano de escolaridade e do ano médio de educação dos adultos. Observando o  gráfico dos Anos de Escolaridade é possível notar que até 50% dos dados possuem aproximadamente até 6 anos de escolaridade e até 75% das observações possuem pouco mais de 10 anos de escolaridade.

- Ao observarmos o gráfico de anos médios de educação para Adultos, é possível notar que esta variável possui uma série de outliers. Observando este boxplot podemos perceber que até 75% das observações tem aproximadamente um valor de 12 para os anos médios de escolaridade.

In [ ]:
 df_tot = (
    df_eda
    .groupby('hogar_total', as_index=False)
    .count()
    .loc[:,['hogar_total', 'Id']]
    .rename(columns={'Id': 'qtd'})
)   

fig, ax = plt.subplots(figsize=(10, 7))
fig.suptitle('Percentual de Casas por Número Total de Moradores', fontsize=16)

sns.barplot(ax=ax, x='hogar_total', y='qtd', data=df_tot, palette='flare')
add_porcentagem(ax, df_tot.qtd.sum())

- O gráfico acima apresenta o percentual de casas por número de moradores. A partir deste gráfico podemos observar que 26,1% das famílias observadas possuem 4 moradores e 22,9% possuem 3 moradores. Outro ponto importante é que uma pequena parte dos dados é composta por famílias que possuem de 9 a 13 moradores.

In [ ]:
################## Agrupando Por Jovens e Crianças ######################################
col = 'hogar_nin'

def range_col(val):
    if val >= 0 and val <= 2:
        return '0 a 2'
    elif val >= 3 and val <= 4:
        return '3 a 4'
    else:
        return '5 a 10'

# Agrupando por "hogar_nin"
df_per = (
    df_eda
    .groupby(col, as_index=False)
    .count()
    .loc[:,[col, 'Id']]
    .rename(columns={'Id': 'qtd'})
)

# Criando ranges "hogar_nin"
df_per['range'] = df_per[col].apply(range_col)

# Somando valores do "range"
df_per = (
    df_per
    .groupby('range', as_index=False)
    .sum()
    .loc[:,['range', 'qtd']]
)

# Adicionado valor percentual ao "range"
df_per['range_lbl'] = round(df_per.qtd/ df_per.qtd.sum() * 100, 1)
df_per['range_lbl'] = df_per['range'] + ' (' + df_per['range_lbl'].astype(str) + '%)'



########## Agrupando por Senhores #############################################################

color=['red','#1C9FB0','#32A0CE','#1C51B0']
col = 'hogar_mayor'

def range_col(val):
    if val >= 2:
        return '2 a 5'
    return str(val)

# Agrupando por "hogar_mayor"
df_per2 = (
    df_eda
    .groupby(col, as_index=False)
    .count()
    .loc[:,[col, 'Id']]
    .rename(columns={'Id': 'qtd'})
)
# Criando ranges "hogar_mayor"
df_per2['range'] = df_per2[col].apply(range_col)
# Somando valores do "range"
df_per2 = (
    df_per2
    .groupby('range', as_index=False)
    .sum()
    .loc[:,['range', 'qtd']]
)

# Adicionado valor percentual ao "range"
df_per2['range_lbl'] = round(df_per2.qtd / df_per2.qtd.sum() * 100, 1)
df_per2['range_lbl'] = df_per2['range'] + ' (' + df_per2['range_lbl'].astype(str) + '%)'


######## Agrupando por Adultos #############################################################

color=['red','#1C9FB0','#32A0CE','#1C51B0']
col = 'hogar_adul'

def range_col(val):
    if val == 0 or val == 1:
        return '0 a 1'
    elif val == 2:
        return '2'
    elif val == 3:
        return '3'
    elif val == 4:
        return '4'
    else:
        return '5 a 9'

# Agrupando por "hogar_adul"
df_per3 = (
    df_eda
    .groupby(col, as_index=False)
    .count()
    .loc[:,[col, 'Id']]
    .rename(columns={'Id': 'qtd'})
)

#"""
# Criando ranges "hogar_adul"
df_per3['range'] = df_per3[col].apply(range_col)

# Somando valores do "range"
df_per3 = (
    df_per3
    .groupby('range', as_index=False)
    .sum()
    .loc[:,['range', 'qtd']]
)

# Adicionado valor percentual ao "range"
df_per3['range_lbl'] = round(df_per3.qtd / df_per3.qtd.sum() * 100, 1)
df_per3['range_lbl'] = df_per3['range'] + ' (' + df_per3['range_lbl'].astype(str) + '%)'


#### =======================================================================

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
fig.suptitle('Famílias por Número de Indivíduos', fontsize=16)

#color = []
squarify.plot(ax=ax[0], sizes=df_per.qtd, label=df_per.range_lbl, alpha=0.6, color=['#ff8d00', '#eba144', '#ffc47a'])
ax[0].set_title('Crianças e Adolescentes com Idade de 0 a 19', pad=20)
ax[0].axis('off')

squarify.plot(ax=ax[1], sizes=df_per2.qtd, label=df_per2.range_lbl, alpha=0.6, color=['#00ff5e', '#2bd66a', '#6bffa2'])
ax[1].set_title('Senhores com Idade acima de 65 anos', pad=20)
ax[1].axis('off')

squarify.plot(ax=ax[2], sizes=df_per3.qtd, label=df_per3.range_lbl, alpha=0.6, color=['#3d48ba', '#3c00ff', '#5f6dfa', '#828dff', '#a8b0ff', '#ccd1ff'])
ax[2].set_title('Adultos', pad=20)
ax[2].axis('off')

plt.tight_layout()
plt.show()

- O gráfico acima apresenta o número de famílias agrupado pelo número e idade de seus moradores.

- No gráfico laranjado podemos observar que 81,04% das famílias observadas possuem de 0 a 2 crianças e/ou adolescentes e que 2,7% dos dados são de casas que possuem de 5 a 10 crianças e/ou adolescentes.

- No gráfico verde é possível observar que 78,1% das residências não possuem nenhum idoso com mais de 65 anos. Mas, 6,9% dos dados possuem de 2 a 5 idosos em suas moradias.

- Já no gráfico azul é possível observar que a maior parte do número de observações é composta por famílias que possuem 2 adultos (46,2%). Já, a menor parte dos dados são de casas que possuem de 5 a 9 adultos (6,6%).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
pers = [df_eda.male.sum(), df_eda.female.sum()]
circle=plt.Circle( (0,0), 0.65, color='white')

ax.pie(
    pers, labels=['Homens', 'Mulheres'],
    startangle=90, autopct='%1.1f%%', pctdistance=.83, colors=['#8017d1', '#a36ecc'],
    textprops={'fontsize': 14}
)
ax.add_artist(circle)
plt.show()

- O gráfico acima apresenta o percentual de homens e mulheres no dataset. Podemos observar que as mulheres representam 51,2% dos dados observados e os homens representam 48,8% destes dados.

## Análise Bivariada

In [ ]:
df_mean = (
    df_eda
    .groupby('tamhog', as_index=False)
    .agg({'v2a1': 'mean'})
    .assign(v2a1=lambda df: round(df.v2a1, 2))
)
fig, ax = plt.subplots(figsize=(10, 6))
ax = sns.barplot(y='tamhog', x='v2a1', data=df_mean, palette='flare', orient='h')
fig.suptitle('Pagamento do Aluguel Mensal (v2a1) pelo tamanho da Casa (tamhog)', fontsize=16)

def add_val(ax):
    for p in ax.patches:
        val = p.get_width()
        x = p.get_x() + p.get_width()
        y = p.get_y() + p.get_height() - .2
        ax.annotate(val, (x, y), size = 12)
    plt.show()

add_val(ax)

- O gráfico acima aprensenta a média do valor do aluguel pelo tamanho das casas. Podemos observar que as casas com menor tamanho e maior tamanho nem sempre são as mais baratas ou mais caras. Casas com tamanho de 11 possuem média do valor mensal de aluguel de 185273.13, já, as casas com valor mensal mais baixo são casas de tamanho 13 com média do valor do aluguel igual a 149023.13.

In [ ]:
df_ha = (
    df_eda
    .groupby(['hacdor', 'hacapo'], as_index=False)
    .count()
    .loc[:, ['hacdor', 'hacapo', 'Id']]
)

def range_ha(row):
    if row.hacdor == 0 and row.hacapo == 0:
        return 'Sem Superlotação'
    elif row.hacdor == 0 and row.hacapo == 1:
        return 'Superlotação de Salas'
    elif row.hacdor == 1 and row.hacapo == 0:
        return 'Superlotação de Quartos'
    else:
        return 'Superlotação de Quartos e Salas'

df_ha['lbl'] = df_ha.apply(range_ha, axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
fig.suptitle('Casas com Superlotação em Salas e Quartos', fontsize=16)
pie_chart(df_ha.Id, df_ha.lbl, ax)
plt.show()

- O gráfico acima apresenta as casas com superlotações. Podemos perceber que 94,8% das casas observadas não possuem superlotação. Já, 2,3% das casas possuem supelotação nos quartos e salas simultaneamente.

In [ ]:
df_cs = (
    df_eda
    .groupby(['cielorazo', 'sanitario1'], as_index=False)
    .count()
    .loc[:, ['cielorazo', 'sanitario1', 'Id']]
    .rename(columns={'Id': 'qtd'})
)

def range_ha(row):
    if row.cielorazo == 0 and row.sanitario1 == 0:
        return 'Sem piso, Com banheiro'
    elif row.cielorazo == 0 and row.sanitario1 == 1:
        return 'Sem piso, Sem banheiro'
    elif row.cielorazo == 1 and row.sanitario1 == 0:
        return 'Com piso, Com banheiro'
    else:
        return 'Com piso, Sem banheiro'

df_cs['label'] = df_cs.apply(range_ha, axis=1)

fig, ax = plt.subplots(figsize=(10, 7))
fig.suptitle('Casas com Piso e Banheiro', fontsize=16)

sns.barplot(ax=ax, x='label', y='qtd', data=df_cs, palette='flare')
add_porcentagem(ax, df_cs.qtd.sum(), w=.5, h=150)

- O gráfico acima apresenta o percentual de casas com/sem piso e com/sem banheiro. A partir deste gráfico podemos observar que 67,1% das casas observadas possuem piso e banheiro, 32,5% não possuem piso mas possuem banheiro e 0,3% das casas não possuem nem piso nem banheiro.

In [ ]:
cols_float = []
for col in df_all.columns:
    if df_all[col].dtype == float:
        cols_float.append(col)

cols_float.append('Target')

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
corr = df_eda.corr('spearman')
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(corr, ax=ax, mask=mask, annot=True)
ax.set_title('Matriz de Correlação com Variáveis Quantitativas da Análise Exploratória', fontsize=16)
plt.show()

- No gráfico acima foi plotado a matriz de correlação das variáveis quantitativas utilizadas na análise exploratória dos dados. A partir deste gráfico podemos observar algumas correlações relavantes, como por exemplo:
    - "rooms" x "cielorazo" (baixa correlação - 0.32) - Quanto maior o número de cômodos maior a probabilidade da casa possuir piso.
    - "rooms" x "bedrooms" (correlação forte - 0.81) - Quanto maior o número de cômodos maior o número de quartos para dormir.
    - "hogar_nin" x "hogar_total" (correlação forte - 0.72) - Quanto maior o número de crianças e jovens maior o número total de pessoas nas casas.
    - "tamhog" x "hogar_adul" (correlação moderada - 0.56) - Quanto maior o tamanho da casa maior o número de adultos com idade entre 20 e 64 anos.

- Para geração deste gráfico foi utilizado o coeficiente de correlação de spearman devido a existências de variáveis contínuas e discretas.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
corr = df_train[cols_float].corr('spearman')
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(corr, ax=ax, mask=mask, annot=True)
ax.set_title('Matriz de Correlação com Outras Variáveis Quantitativas', fontsize=16)

plt.show()

- No gráfico acima foi plotado a matriz de correlação com outras variáveis quantitativas presentes na base de dados. A partir deste gráfico podemos observar algumas correlações relevantes, como por exemplo:
    - "meaneduc" x "v2a1" - (correlação moderada - 0.47) Quanto maior a média de anos de educação dos adultos maior é o valor do aluguel pago mensalmente.
    - "v2a1" x "v18q1" - (correlação moderada - 0.32) Quanto maior o valor do aluguel pago mensal, maior o número de tablets nas residências.

- A variáveis que possuem o prefixo SQB não foram citadas pois, estas features são valores ao quadrado de outras variáveis: Exemplo: "SQBovercrowding" é o valor de "overcrowding" ao quadrado.

In [ ]:
pair = sns.pairplot(df_train[cols_float], corner=True)
pair.fig.suptitle('Scatterplots Entre as Variáveis Quantitativas', fontsize=26)
plt.show()

- O gráfico acima apresenta os scatterplots de algumas quantitativas presentes na base de dados. Observando o comportamento dos dados de forma visual (com excessão das colunas "meaneduc" x "SQBmeaned" e "overcrowding" x "SQBovercrowding") podemos notar que não existem fortes correlações e alinhamento dos dados.

## Exportando dados Corrigidos

In [ ]:
df_train = df_all.query('df_type == "train"').merge(target, on='Id').copy()
df_train.drop(columns=['df_type'], inplace=True)
df_train.to_csv('new_train.csv', index=False)

In [ ]:
df_test = df_all.query('df_type == "test"').copy()
df_test.drop(columns=['df_type'], inplace=True)
df_test.to_csv('new_test.csv', index=False)